In [ ]:
%%bash 
rsync -r LRMfwonLocalhost:/home/debian/test . 

In [1]:
import sys
sys.path.insert(0, "/home/esr/LRMessanlage/pyLRM")
from pathlib import Path 
import parse
import pandas as pd
import datetime
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import DatetimeTickFormatter

from logging_handler import ax_sensor_log_parser,_ax_sensor_log_row_parser
output_notebook()


def calc_pby(df,time_threshold_s):
    """
    diese funktion splittet eine df von achsdaten in mehrere unterlisten falls 
    die Zeit zwichen Achsdaten Ereignisse > ist als den angegebene time_threshold_s
    """
    pby = []
    pbycnt=0
    ax = []
    axcnt=0
    for toff in df.time_wheel_off.values:
        if toff> time_threshold_s:
            axcnt=0
            pbycnt+=1
        else:
            axcnt+=1
        pby.append(pbycnt)
        ax.append(axcnt)
    return df.assign(passby=pby).assign(ax=ax)

Loading BokehJS ...

### Messung Pfad und name  

In [2]:
path= Path("/home/esr/Documents/Luzern/")
mess_p=Path("/home/esr/Documents/Luzern/Nachmessung_23-11/")
log_p=mess_p.joinpath("LRM_messung.log")
rec_p = path.joinpath("KG")
print("{} exists: {}.".format(log_p.as_posix(),log_p.exists()))

/home/esr/Documents/Luzern/Nachmessung_23-11/LRM_messung.log exists: True.


### Lade achs data
die achs daten sind vom LRM_messung.log abgeholt.


In [3]:
ax_data = pd.DataFrame(ax_sensor_log_parser(log_p)['MSG_HEADER_AXLE']).T

### Filtern anahand daten intervalle

In [4]:
zeitintervalle = [
    (pd.Timestamp("2018-11-13 07:28:00"),pd.Timestamp("2018-11-16 22:00:00")),
                 ]

zeitintervalle = [
    (pd.Timestamp("2018-11-2 "),pd.Timestamp("2018-11-11 22:00:00")),
                 ]

zeitintervalle = [
    (pd.Timestamp("2018-11-19 "),pd.Timestamp("2018-11-26 22:00:00")),
                 ]

#between_times= ['8:00', '12:00']

In [5]:
ax_df = pd.concat([ax_data[ti_start:ti_stop] for (ti_start,ti_stop) in zeitintervalle])
#ax_df = ax_df.between_time(*between_times)
ax_df.index.name="timestamp"

### Plot

In [7]:
p = figure(x_axis_type="datetime", plot_width=1500, plot_height=750,tools=['xwheel_zoom','xpan'], active_scroll='xwheel_zoom',active_drag='xpan')
p.scatter('timestamp', 'ax_number',marker='circle',alpha=0.5,size=5, source=ColumnDataSource(ax_df), color="orange")


p.xaxis.formatter = DatetimeTickFormatter(seconds=["%H:%M:%S"],
                                            minutes=["%H:%M"],
                                            minsec=["%H:%M:%S"],
                                            hours=["%H"])
show(p)

### Plot2
falls die geloggte achzahlen nicht in passby unterteilt wurden kann mandie funktion `calc_pby` angewenden um passby zu unterscheiden

In [13]:
df_km94_2 =ax_df[ax_df.from_axle_sensor=="km94_2"].sort_index()
df_km94_4 =ax_df[ax_df.from_axle_sensor=="km94_4"].sort_index()
pb_km94_2=calc_pby(df_km94_2,10)
pb_km94_4=calc_pby(df_km94_4,10)

In [14]:
p = figure(x_axis_type="datetime", plot_width=1500, plot_height=750,tools=['xwheel_zoom','xpan'], active_scroll='xwheel_zoom',active_drag='xpan')
p.scatter('timestamp', 'ax',marker='circle',alpha=0.5,size=5, source=ColumnDataSource(pb_km94_2),legend="km94_2(906D)", color="orange")
p.scatter('timestamp', 'ax',marker="square",alpha=0.5,size=5,source=ColumnDataSource(pb_km94_4),legend="km94_4(3136)(SSA)", color="green")


p.xaxis.formatter = DatetimeTickFormatter(seconds=["%H:%M:%S"],
                                            minutes=["%H:%M"],
                                            minsec=["%H:%M:%S"],
                                            hours=["%H"])
show(p)

### Teste assignement

assignement zwischen passby und XLR Daten  

In [12]:
import XL2rec_to_passby_assignement as a
import logging
logger = logging

def filt_rec_passby_list(rp_list,zeitintervalle):
    list_f = []
    for pb in rp_list:
        for (t0,t1) in zeitintervalle:
            if pb['start_rec']>t0 and pb['stop_rec']<t1:
                list_f.append(pb)
                break
    return list_f
    

## Lade passby daten

In [28]:
_,passby_list=a.load_passby_info_from_dir(mess_p,logger,name_match ="*.json")

## Filtere liste anhand datenintervalle 

In [29]:
passby_list_f = filt_rec_passby_list(passby_list,zeitintervalle)
print("Remaining passby: {}.".format(len(passby_list_f)))

Remaining passby: 419.


In [11]:
from bokeh.models import BoxAnnotation
for pb in passby_list_f:
    low_box = BoxAnnotation(left=pb['start_rec'], right=pb['stop_rec'], fill_color="Blue")
    p.add_layout(low_box)
show(p)

In [31]:
def test_passby_time_correction(passby_list,outlier_th=50):
    out=[]
    approx_time_correction = [p['xl2_time_correction'].total_seconds() for p in passby_list]
    print(pd.Series(approx_time_correction).describe())
    avg = pd.Series(approx_time_correction).mean()
    for tc,p in zip(approx_time_correction,passby_list):
        if abs(tc-avg) >outlier_th:
                print("time correction: {},  passby {} ".format(tc,p['path']))
                out.append(p['path'])

    return passby_list[0]['xl2_time_correction'],out

time_correction, outlier = approx_time_correction(passby_list_f,outlier_th=10)
print(time_correction.total_seconds())

count     419.000000
mean    -7233.003978
std         0.609071
min     -7234.494179
25%     -7233.540743
50%     -7233.002012
75%     -7232.522106
max     -7231.870063
dtype: float64
-7232.083072


In [32]:
import os
for o in outlier:
    #os.remove(o.as_posix())

## Lade XLR rcords 

In [33]:
rec_list = a.load_xl2logs_from_dir(rec_p,logger)

In [34]:
a.max_duration(rec_list,logger,max_duration=120)

[]

In [35]:
rec_list_f = filt_rec_passby_list(rec_list,[(t0-time_correction,t1-time_correction) for (t0,t1) in zeitintervalle])
a.max_duration(rec_list_f,logger)
print("Remaining passby: {}.".format(len(rec_list_f)))

Remaining passby: 1063.


In [42]:
from bokeh.models import BoxAnnotation
for r in rec_list_f:
    low_box = BoxAnnotation(left=r['start_rec']+time_correction, right=r['stop_rec']+time_correction, fill_color="red")
    p.add_layout(low_box)
show(p)

AttributeError: 'dict' object has no attribute 'add_layout'

## Assign

In [36]:
assigned,not_assigned,not_correctly_assigned=[],[],[]
used_recs =[]
for p in passby_list_f:
    assigned_recs = a.assign_xl2rec_to_passby(**p, xl2_records=rec_list_f,logger=logger)
    n_assignements = len(assigned_recs)
    if n_assignements == 0:
        not_assigned.append(p)
        logger.warning("Not assigned: {}, {}.".format(p['path'].parent.parent.name, p['path'].name))
    elif n_assignements == 1:
        p['xl2_rec'] = assigned_recs
        used_recs.append(*assigned_recs)
        assigned.append(p)
    elif n_assignements > 1:
        not_correctly_assigned.append(p)
        logger.warning("Not correctlyassigned: {}, {}.".format(p['path'].parent.parent.name, p['path'].name))
        
logger.info("Correctly assigned {}".format(len(assigned)))
logger.warning("Not assigned {}".format(len(not_assigned)))
logger.warning("Not correctly assigned {}".format(len(not_correctly_assigned)))

In [37]:
len(assigned)

415

In [17]:
used_recs_p = [r['path'] for r in used_recs]

In [18]:
len(set(used_recs_p))

693

In [34]:
len(set(used_recs_p)

693

In [19]:
all_recs_p = set([r['path'] for r in rec_list_f])

In [21]:
[a.load_xl2logs_info(p) for p in all_recs_p.difference(used_recs_p)]

[{'stop_rec': datetime.datetime(2018, 11, 15, 4, 51, 52),
  'start_rec': datetime.datetime(2018, 11, 15, 4, 51, 38),
  'path': PosixPath('/home/esr/Documents/Luzern/XL2KG/2018-11-15_SLM_026_123_Log.txt')},
 {'stop_rec': datetime.datetime(2018, 11, 13, 21, 54, 47),
  'start_rec': datetime.datetime(2018, 11, 13, 21, 54, 34),
  'path': PosixPath('/home/esr/Documents/Luzern/XL2KG/2018-11-13_SLM_067_123_Log.txt')},
 {'stop_rec': datetime.datetime(2018, 11, 14, 20, 52, 4),
  'start_rec': datetime.datetime(2018, 11, 14, 20, 51, 22),
  'path': PosixPath('/home/esr/Documents/Luzern/XL2KG/2018-11-14_SLM_183_123_Log.txt')},
 {'stop_rec': datetime.datetime(2018, 11, 14, 12, 12, 49),
  'start_rec': datetime.datetime(2018, 11, 14, 12, 12, 36),
  'path': PosixPath('/home/esr/Documents/Luzern/XL2KG/2018-11-14_SLM_014_123_Log.txt')},
 {'stop_rec': datetime.datetime(2018, 11, 13, 20, 43, 31),
  'start_rec': datetime.datetime(2018, 11, 13, 20, 43, 18),
  'path': PosixPath('/home/esr/Documents/Luzern/XL2K